In [1]:
#importing dependecies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


#import input dataset
attrition_df= pd.read_csv('HR-Employee-Attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


Start Preprocessing.

In [2]:
#generate categorical variable list to encode later
attrition_cat= list(attrition_df.dtypes[attrition_df.dtypes=='object'].index)

attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [3]:
#check number of unique values in each column. more than 10 variables would requirek bucketing
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [7]:
#encode more than one variable using the attrition_cat variable list

#create a OneHotEncoder instance
enc= OneHotEncoder(sparse=False)

#fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

#add the encoded variable names to the dataframe
encode_df.columns= enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
#merge OneHotEncoded features and drop the originals
attrition_df= attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df= attrition_df.drop(attrition_cat, 1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [11]:
#split the preprocessed data into our features and target arrays
y= attrition_df['Attrition_Yes'].values
X = attrition_df.drop(["Attrition_Yes","Attrition_No"],1).values

#split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=78)

In [12]:
#Create a standard scaler instance
scaler= StandardScaler()

#fit the standard scaler
X_scaler= scaler.fit(X_train)

#scale the data
X_train_scaled= X_scaler.transform(X_train)
X_test_scaled= X_scaler.transform(X_test)

Preprocessing complete. start defining the model

In [15]:
#define the model - deep neural net
number_input_features= len(X_train[0])
hidden_nodes_layer1= 8
hidden_nodes_layer2= 5

nn= tf.keras.models.Sequential()

#first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

#second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

#output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#check the structure of hte model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [16]:
#compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

defining model complete. start training the model

In [17]:
#train the model
fit_model= nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 0s 881us/step - loss: 756.4064 - accuracy: 0.1759
Epoch 2/100
35/35 [==============================] - 0s 851us/step - loss: 296.4540 - accuracy: 0.2331
Epoch 3/100
35/35 [==============================] - 0s 851us/step - loss: 54.3038 - accuracy: 0.6935
Epoch 4/100
35/35 [==============================] - 0s 792us/step - loss: 25.5205 - accuracy: 0.7527
Epoch 5/100
35/35 [==============================] - 0s 792us/step - loss: 16.3855 - accuracy: 0.7289
Epoch 6/100
35/35 [==============================] - 0s 763us/step - loss: 10.4954 - accuracy: 0.7780
Epoch 7/100
35/35 [==============================] - 0s 763us/step - loss: 7.4664 - accuracy: 0.7696
Epoch 8/100
35/35 [==============================] - 0s 821us/step - loss: 5.3395 - accuracy: 0.8043
Epoch 9/100
35/35 [==============================] - 0s 733us/step - loss: 4.2504 - accuracy: 0.7632
Epoch 10/100
35/35 [==============================] - 0s 704us/step - loss: 3.5455 

In [18]:
#evaluate the model using test data
model_loss, model_accuracy= nn.evaluate(X_test, y_test, verbose=2)
print(f'loss: {model_loss}, accuracy: {model_accuracy}')

12/12 - 0s - loss: 0.3838 - accuracy: 0.8641
loss: 0.383795827627182, accuracy: 0.864130437374115
